In [6]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, Dropout, Flatten, Dense, GlobalAveragePooling2D

In [7]:
def fire_module(x, squeeze, expand):
    squeeze = Conv2D(filters=squeeze, kernel_size=(1, 1), activation='relu', padding='same')(x)
    expand_1x1 = Conv2D(filters=expand, kernel_size=(1, 1), activation='relu', padding='same')(squeeze)
    expand_3x3 = Conv2D(filters=expand, kernel_size=(3, 3), activation='relu', padding='same')(squeeze)
    output = Concatenate(axis=-1)([expand_1x1, expand_3x3])
    return output

In [8]:
def SqueezeNet(input_shape=(224,224,3), num_classes=10):
    input_tensor = Input(shape=input_shape)
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu', padding='valid')(input_tensor)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = fire_module(x, squeeze=16, expand=64)
    x = fire_module(x, squeeze=16, expand=64)
    x = fire_module(x, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = fire_module(x, squeeze=32, expand=128)
    x = fire_module(x, squeeze=48, expand=192)
    x = fire_module(x, squeeze=48, expand=192)
    x = fire_module(x, squeeze=64, expand=256)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = fire_module(x, squeeze=64, expand=256)
    x = Dropout(0.5)(x)
    x = Conv2D(filters=num_classes, kernel_size=(1, 1), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output_tensor = Dense(units=num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

In [9]:
squeeze_net = SqueezeNet()
squeeze_net.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_26 (Conv2D)             (None, 111, 111, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 55, 55, 64)  0           ['conv2d_26[0][0]']              
                                                                                              

As you can see in the first line of the function, using the Keras Functional API, ```tf.keras.layers.Conv2D()``` instantiates a convolutional layer which is then called with the input x. We can slightly transform the fire() function so that it uses the same semantics:

In [10]:
def fire(x, squeeze, expand):
  y = Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
  y1 = Conv2D(filters=expand, kernel_size=1, activation='relu', padding='same')(y)
  y3 = Conv2D(filters=expand, kernel_size=3, activation='relu', padding='same')(y)
  return Concatenate(axis=-1)([y1, y3])

In [11]:
def fire_module(squeeze, expand):
  return lambda x: fire(x, squeeze, expand)

In [12]:
def SqueezeNet(input_shape=(224,224,3), num_classes=10):
  input_tensor = Input(shape=input_shape)
  x = Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu', padding='valid')(input_tensor)
  x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

  x = fire_module(squeeze=16, expand=64)(x)
  x = fire_module(squeeze=16, expand=64)(x)
  x = fire_module(squeeze=32, expand=128)(x)
  x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

  x = fire_module(squeeze=32, expand=128)(x)
  x = fire_module(squeeze=48, expand=192)(x)
  x = fire_module(squeeze=48, expand=192)(x)
  x = fire_module(squeeze=64, expand=256)(x)
  x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

  x = fire_module(squeeze=64, expand=256)(x)

  x = Dropout(0.5)(x)
  x = Conv2D(filters=num_classes, kernel_size=(1, 1), activation='relu', padding='same')(x)
  x = GlobalAveragePooling2D()(x)
  output_tensor = Dense(units=num_classes, activation='softmax')(x)

  model = Model(inputs=input_tensor, outputs=output_tensor)

  return model

In [13]:
squeeze_net = SqueezeNet()
squeeze_net.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_52 (Conv2D)             (None, 224, 224, 32  896         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_53 (Conv2D)             (None, 224, 224, 16  528         ['conv2d_52[0][0]']              
                                )                                                           